In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [27]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [28]:
%%capture pwd
!pwd

In [29]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-08-04-where-are-the-uks-levelling-up-funds-most-needed', 300, 500)

# Fig 1

In [30]:
df = pd.read_csv("raw/tracktheeconomyacuk_england_index_db.csv")

In [31]:
f = "fig1_imd"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,local_authorities,y_axis_allcat_rank,x_axis_imd_rank,lad_id,country,levelling_up_categorisation,region,supergroup_name,quadrant,label_2_highest_lowest
0,Haringey,11,37,E09000014,England,2,London,London Cosmopolitan,quadrant 1,lab
1,Great Yarmouth,26,24,E07000145,England,1,East,Countryside Living,quadrant 1,lab
2,Luton,8,52,E06000032,England,1,East,Ethnically Diverse Metropolitan Living,quadrant 1,nolab
3,Tendring,29,32,E07000076,England,1,East,Countryside Living,quadrant 1,nolab
4,Barking and Dagenham,58,5,E09000002,England,1,London,Ethnically Diverse Metropolitan Living,quadrant 1,nolab


In [277]:
base = (
    alt.Chart(f1)
    .encode(
        tooltip="tooltip:N",
        x=alt.X(
            "x_axis_imd_rank:Q",
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        y=alt.Y(
            "y_axis_allcat_rank:Q",
            sort=[],
            axis=None,
            scale=alt.Scale(domain=[0, 300]),
        ),
        fill=alt.Color(
            "supergroup_name:N",
            scale=alt.Scale(
                range=[
                    colors["eco-red"],
                    'white',
                    colors["eco-mid-blue"],
                    'white',
                    colors["eco-green"],
                    'white',
                    colors["eco-gray"],
                    'white',
                ],
            ),
        ),
        color=alt.Color(
            "supergroup_name:N",
            legend=alt.Legend(title="", labelLimit=400, orient="bottom", columns=2),
            scale=alt.Scale(
                range=[
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-red"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-mid-blue"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-green"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7),
                    scale_lightness(colors["eco-gray"], 0.7)
                ],
            ),
        ),
        shape=alt.Shape(
            "region:N",
            legend=alt.Legend(
                title="",
                labelLimit=170,
                orient="bottom",
                columns=3,
                symbolFillColor=colors["eco-gray"],
                symbolStrokeColor=scale_lightness(colors["eco-gray"], 0.7),
                offset=30,
            ),
            scale=alt.Scale(
                range=['triangle-right','cross','circle',
                       'triangle-up','diamond','triangle-down',
                       'triangle-left',"M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z",'square',
                      ]
            )
        ),
    )
    .transform_calculate(
        tooltip="datum.region+' 🏨 '+datum.local_authorities+' 📊 X: '+datum.x_axis_imd_rank+'. Y: '+datum.y_axis_allcat_rank+'.'"
    )
)
points1 = base.mark_point(size=100, opacity=0.8).encode()
label1=alt.Chart(pd.DataFrame([{'x':0,'y':-10,'t':'⬅ Increasing deprivation'}])).mark_text(
    size=11,color=colors['eco-gray'],align='left'
).encode(x='x',y='y',text='t')
label2=alt.Chart(pd.DataFrame([{'x':-10,'y':0,'t':'⬅ Increasing COVID economic impact'}])).mark_text(
    size=11,color=colors['eco-gray'],align='left',angle=270
).encode(x='x',y='y',text='t')
label3=alt.Chart(pd.DataFrame([{'x':310,'y':320,'t':'Decreasing deprivation ➡'}])).mark_text(
    size=11,color=colors['eco-gray'],align='right'
).encode(x='x',y='y',text='t')
label4=alt.Chart(pd.DataFrame([{'x':320,'y':310,'t':'Decreasing COVID economic impact ➡'}])).mark_text(
    size=11,color=colors['eco-gray'],align='right',angle=270
).encode(x='x',y='y',text='t')
line1 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(x="x:Q")
)
line2 = (
    alt.Chart(pd.DataFrame([{"x": 155}]))
    .mark_rule(strokeDash=[5, 3], color=colors["eco-gray"])
    .encode(y="x:Q")
)
area1 = (
    alt.Chart(
        pd.DataFrame([{"x": 155, "y": 0, "y2": 311}, {"x": 311, "y": 0, "y2": 311}])
    )
    .mark_area(opacity=0.15, color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors['eco-gray'], offset=0.3
                    ),
                ],
                x1=1,
                x2=0,
                y1=1,
                y2=1,
            ))
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
area2 = (
    alt.Chart(
        pd.DataFrame([{"x": 0, "y": 155, "y2": 330}, {"x": 311, "y": 155, "y2": 330}])
    )
    .mark_area(opacity=0.15, color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors['eco-gray'], offset=0.3
                    ),
                ],
                x1=1,
                x2=1,
                y1=0,
                y2=1,
            ))
    .encode(x="x:Q", y="y:Q", y2="y2:Q")
)
layer1 = (
    ((area1+area2+points1 + line1 + line2+label1+label2+label3+label4).properties(height=400, width=600))
    .configure_view(stroke=None)
    .properties(title="")
)
# layer1.save("visualisation/" + f + ".json")
# layer1.save("visualisation/" + f + ".png")
# open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [153]:
df = pd.read_excel(
    "raw/figures.xlsx", sheet_name="Fig2_Fig3", skiprows=2, usecols="B:F", nrows=10
)
df = df.set_index(df.columns[0]).stack().reset_index()
df.columns = ["advantage", "age", "value"]
df["value"] /= 100
df = df[::-1]

In [154]:
f = "fig2_advantages"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,advantage,age,value
39,Improved work-life balance,50-69,0.64
38,Improved work-life balance,30-49,0.67
37,Improved work-life balance,16-29,0.62
36,Improved work-life balance,Overall,0.65
35,Fewer distractions,50-69,0.48


In [155]:
bars = (
    alt.Chart(f2)
    .encode(
        x=alt.X(
            "value:Q",
            stack=False,
            title="",
            axis=alt.Axis(
                grid=False,
                title="",
                titleAnchor="end",
                titleX=415,
                titleY=7,
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        y=alt.Y("advantage:N", title="", axis=None, sort=[]),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.advantage+' 🏢 '+datum.age+' 📈 '+round(datum.value*100)+'%'"
    )
)

bars1 = bars.mark_bar(
    size=11, yOffset=-14, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.age=='16-29'")
bars2 = bars.mark_bar(
    size=11, yOffset=-26, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.age=='Overall'")
bars3 = bars.mark_bar(
    size=11, yOffset=-2, color=colors["eco-light-blue"], opacity=0.8
).transform_filter("datum.age=='30-49'")
bars4 = bars.mark_bar(
    size=11, yOffset=10, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.age=='50-69'")
labels1 = bars1.mark_text(
    align="left",
    xOffset=5,
    yOffset=-14,
    color=colors["eco-turquiose"],
).encode(text=alt.Text("value:N", format=".0%"))
labels2 = bars2.mark_text(
    align="left",
    xOffset=5,
    yOffset=-26,
    color=colors["eco-green"],
).encode(text=alt.Text("value:N", format=".0%"))
labels3 = bars3.mark_text(
    align="left",
    xOffset=5,
    yOffset=-2,
    color=colors["eco-light-blue"],
).encode(text=alt.Text("value:N", format=".0%"))
labels4 = bars4.mark_text(
    align="left",
    xOffset=5,
    yOffset=10,
    color=colors["eco-mid-blue"],
).encode(text=alt.Text("value:N", format=".0%"))
titles = (
    bars.mark_text(
        align="left", size=11, baseline="bottom", yOffset=-35, color=colors["eco-gray"]
    )
    .encode(
        text="advantage:N",
        x=alt.X("x0:Q", axis=alt.Axis(format=".0%")),
        y=alt.Y("advantage:N", sort=[]),
    )
    .transform_filter("datum.age=='Overall'")
    .transform_calculate(x0="0")
)
legendkey = "Fewer distractions"
legendx = 0.7
legendfs = 11
legend1 = (
    bars1.mark_text(
        align="right", size=legendfs, yOffset=-14, color=colors["eco-turquiose"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend2 = (
    bars2.mark_text(
        align="right", size=legendfs, yOffset=-27, color=colors["eco-green"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend3 = (
    bars3.mark_text(
        align="right", size=legendfs, yOffset=0, color=colors["eco-light-blue"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend4 = (
    bars4.mark_text(
        align="right", size=legendfs, yOffset=13, color=colors["eco-mid-blue"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)

layer = (
    (
        (bars1 + legend1)
        + (bars2 + legend2)
        + (bars3 + legend3)
        + (bars4 + legend4)
        + labels1
        + labels2
        + labels3
        + labels4
        + titles
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=alt.Step(75), width=400)
)
layer.save("visualisation/" + f + ".json")
layer.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [156]:
df = pd.read_excel(
    "raw/figures.xlsx", sheet_name="Fig2_Fig3", skiprows=15, usecols="B:F", nrows=10
)
df = df.set_index(df.columns[0]).stack().reset_index()
df.columns = ["advantage", "age", "value"]
df["value"] /= 100
df = df[::-1]

In [157]:
f = "fig3_disadvantages"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,advantage,age,value
39,Harder to work with others,50-69,0.36
38,Harder to work with others,30-49,0.45
37,Harder to work with others,16-29,0.56
36,Harder to work with others,Overall,0.45
35,More distractions,50-69,0.21


In [158]:
bars = (
    alt.Chart(f3)
    .encode(
        x=alt.X(
            "value:Q",
            stack=False,
            title="",
            axis=alt.Axis(
                grid=False,
                title="",
                titleAnchor="end",
                titleX=415,
                titleY=7,
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        y=alt.Y("advantage:N", title="", axis=None, sort=[]),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.advantage+' 🏢 '+datum.age+' 📈 '+round(datum.value*100)+'%'"
    )
)

bars1 = bars.mark_bar(
    size=11, yOffset=-14, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.age=='16-29'")
bars2 = bars.mark_bar(
    size=11, yOffset=-26, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.age=='Overall'")
bars3 = bars.mark_bar(
    size=11, yOffset=-2, color=colors["eco-light-blue"], opacity=0.8
).transform_filter("datum.age=='30-49'")
bars4 = bars.mark_bar(
    size=11, yOffset=10, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.age=='50-69'")
labels1 = bars1.mark_text(
    align="left",
    xOffset=5,
    yOffset=-14,
    color=colors["eco-turquiose"],
).encode(text=alt.Text("value:N", format=".0%"))
labels2 = bars2.mark_text(
    align="left",
    xOffset=5,
    yOffset=-26,
    color=colors["eco-green"],
).encode(text=alt.Text("value:N", format=".0%"))
labels3 = bars3.mark_text(
    align="left",
    xOffset=5,
    yOffset=-2,
    color=colors["eco-light-blue"],
).encode(text=alt.Text("value:N", format=".0%"))
labels4 = bars4.mark_text(
    align="left",
    xOffset=5,
    yOffset=10,
    color=colors["eco-mid-blue"],
).encode(text=alt.Text("value:N", format=".0%"))
titles = (
    bars.mark_text(
        align="left", size=11, baseline="bottom", yOffset=-35, color=colors["eco-gray"]
    )
    .encode(
        text="advantage:N",
        x=alt.X("x0:Q", axis=alt.Axis(format=".0%")),
        y=alt.Y("advantage:N", sort=[]),
    )
    .transform_filter("datum.age=='Overall'")
    .transform_calculate(x0="0")
)
legendkey = "More distractions"
legendx = 0.58
legendfs = 11
legend1 = (
    bars1.mark_text(
        align="right", size=legendfs, yOffset=-14, color=colors["eco-turquiose"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend2 = (
    bars2.mark_text(
        align="right", size=legendfs, yOffset=-27, color=colors["eco-green"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend3 = (
    bars3.mark_text(
        align="right", size=legendfs, yOffset=0, color=colors["eco-light-blue"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)
legend4 = (
    bars4.mark_text(
        align="right", size=legendfs, yOffset=13, color=colors["eco-mid-blue"]
    )
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='" + legendkey + "'")
    .transform_calculate(x0=str(legendx))
)

layer = (
    (
        (bars1 + legend1)
        + (bars2 + legend2)
        + (bars3 + legend3)
        + (bars4 + legend4)
        + labels1
        + labels2
        + labels3
        + labels4
        + titles
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=alt.Step(75), width=400)
)
layer.save("visualisation/" + f + ".json")
layer.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer

Fontconfig error: Cannot load default config file


alt.LayerChart(...)